# Heart Disease Prediction Notebook

In this notebook we will develop a method to predict heart disease from the heart disease dataset in the UCI Machine Learning repository.

## Importing libraries

In [132]:
import os
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

## Reading data and checking structure

In [161]:
df = pd.read_csv("../data/heart.csv")

In [134]:
df.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [135]:
df.describe()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
count,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000
mean,54.366337,0.683168,0.966997,131.623762,246.264026,0.148515,0.528053,149.646865,0.326733,1.039604,1.399340,0.729373,2.313531,0.544554
std,9.082101,0.466011,1.032052,17.538143,51.830751,0.356198,0.525860,22.905161,0.469794,1.161075,0.616226,1.022606,0.612277,0.498835
min,29.000000,0.000000,0.000000,94.000000,126.000000,0.000000,0.000000,71.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,47.500000,0.000000,0.000000,120.000000,211.000000,0.000000,0.000000,133.500000,0.000000,0.000000,1.000000,0.000000,2.000000,0.000000
50%,55.000000,1.000000,1.000000,130.000000,240.000000,0.000000,1.000000,153.000000,0.000000,0.800000,1.000000,0.000000,2.000000,1.000000
75%,61.000000,1.000000,2.000000,140.000000,274.500000,0.000000,1.000000,166.000000,1.000000,1.600000,2.000000,1.000000,3.000000,1.000000
max,77.000000,1.000000,3.000000,200.000000,564.000000,1.000000,2.000000,202.000000,1.000000,6.200000,2.000000,4.000000,3.000000,1.000000


In [169]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 303 entries, 0 to 302
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       303 non-null    int64  
 1   sex       303 non-null    int64  
 2   cp        303 non-null    int64  
 3   trestbps  303 non-null    int64  
 4   chol      303 non-null    int64  
 5   fbs       303 non-null    int64  
 6   restecg   303 non-null    int64  
 7   thalach   303 non-null    int64  
 8   exang     303 non-null    int64  
 9   oldpeak   303 non-null    float64
 10  slope     303 non-null    int64  
 11  ca        303 non-null    int64  
 12  thal      303 non-null    int64  
 13  target    303 non-null    int64  
dtypes: float64(1), int64(13)
memory usage: 33.3 KB


'None'

Here is a bit more fleshed out description of each variable:

1. **age**: The person's age in years
2. **sex**: The person's sex (1 = male, 0 = female)
3. **cp**: The chest pain experienced (Value 1: typical angina, Value 2: atypical angina, Value 3: non-anginal pain, Value 4: asymptomatic)
4. **trestbps**: The person's resting blood pressure (mm Hg on admission to the hospital)
5. **chol**: The person's cholesterol measurement in mg/dl
6. **bs**: The person's fasting blood sugar (> 120 mg/dl, 1 = true; 0 = false)
7. **restecg**: Resting electrocardiographic measurement (0 = normal, 1 = having ST-T wave abnormality, 2 = showing probable or definite left ventricular hypertrophy by Estes' criteria)
8. **thalach**: The person's maximum heart rate achieved
9. **exang**: Exercise induced angina (1 = yes; 0 = no)
10. **oldpeak**: ST depression induced by exercise relative to rest ('ST' relates to positions on the ECG plot.)
11. **slope**: the slope of the peak exercise ST segment (Value 1: upsloping, Value 2: flat, Value 3: downsloping)
12. **ca**: The number of major vessels (0-3)
13. **thal**: A blood disorder called thalassemia (3 = normal; 6 = fixed defect; 7 = reversable defect)
14. **target**: Heart disease (0 = no, 1 = yes)

## Feature Engineering and Data Cleaning

Let's try to clean the column names of the data. If we name our columns properly, we will avoid confusion in the future during data cleaning or engineering new features.

In [137]:
# Changing column names
df.columns = ['age', 'sex', 'chest_pain_type', 'resting_blood_pressure', 'cholesterol', 'fasting_blood_sugar', 'rest_ecg', 'max_heart_rate_achieved',
       'exercise_induced_angina', 'st_depression', 'st_slope', 'num_major_vessels', 'thalassemia', 'target']

We see that a lot of the features that are supposed to be categorical are in fact encoded as integers. Let's try to clean these up. 

By doing this, we can set up our data for visualizing as well as providing as good foundation for data preprocessing for training models.

In [138]:
def clean_data(df):
    """
    Function will be used to clean the dataset.

    Parameters
    ----------
    df: DataFrame
        A dataframe containing the the heart disease data.
    
    Returns
    -------
    df: DataFrame
        A dataframe containing the cleansed heart disease data.
    """
    df['sex'][df['sex'] == 0] = 'female'
    df['sex'][df['sex'] == 1] = 'male'

    df['chest_pain_type'][df['chest_pain_type'] == 0] = 'typical angina'
    df['chest_pain_type'][df['chest_pain_type'] == 1] = 'atypical angina'
    df['chest_pain_type'][df['chest_pain_type'] == 2] = 'non-anginal pain'
    df['chest_pain_type'][df['chest_pain_type'] == 3] = 'asymptomatic'

    df['fasting_blood_sugar'][df['fasting_blood_sugar'] == 0] = 'lower than 120mg/ml'
    df['fasting_blood_sugar'][df['fasting_blood_sugar'] == 1] = 'greater than 120mg/ml'

    df['rest_ecg'][df['rest_ecg'] == 0] = 'normal'
    df['rest_ecg'][df['rest_ecg'] == 1] = 'ST-T wave abnormality'
    df['rest_ecg'][df['rest_ecg'] == 2] = 'left ventricular hypertrophy'

    df['exercise_induced_angina'][df['exercise_induced_angina'] == 0] = 'no'
    df['exercise_induced_angina'][df['exercise_induced_angina'] == 1] = 'yes'

    df['st_slope'][df['st_slope'] == 0] = 'upsloping'
    df['st_slope'][df['st_slope'] == 1] = 'flat'
    df['st_slope'][df['st_slope'] == 2] = 'downsloping'

    df['thalassemia'][df['thalassemia'] == 0] = 'normal'
    df['thalassemia'][df['thalassemia'] == 1] = 'fixed defect'
    df['thalassemia'][df['thalassemia'] == 2] = 'reversable defect'
    df['thalassemia'][df['thalassemia'] == 3] = 'reversable defect'

    return df

In [1]:
def change_dtypes(df):
    """
    Function will be used to convert features to appropriate type.

    Parameters
    ----------
    df: DataFrame
        A dataframe containing the heart disease data.

    Returns
    -------
    df: DataFrame
        A dataframe containing the cleansed heart disease data.
    """
    df['sex'] = df['sex'].astype('object')
    df['chest_pain_type'] = df['chest_pain_type'].astype('object')
    df['fasting_blood_sugar'] = df['fasting_blood_sugar'].astype('object')
    df['rest_ecg'] = df['rest_ecg'].astype('object')
    df['exercise_induced_angina'] = df['exercise_induced_angina'].astype('object')
    df['st_slope'] = df['st_slope'].astype('object')
    df['thalassemia'] = df['thalassemia'].astype('object')
    
    return df


In [140]:
df = clean_data(df)
df = change_dtypes(df)

## Correlations between features

Let's try to see how our features are correlated with each other as well as the target variable.

In [141]:
# Plotly figure for creating correlation:
fig = go.Figure(data=go.Heatmap(z=df.corr(), 
                                x=df.corr().columns, 
                                y=df.corr().columns, 
                                colorscale=px.colors.sequential.Blugrn, 
                                text=df.corr().values, 
                               ))

# Updating the dimensions and title:
fig.update_layout(height=600, 
                  width=600, 
                  title="Correlation Matrix Heatmap")
fig.show()

There seems to be weak to moderate correlations between the features. Therefore, we can retain all of the features for the purpose of traning.

## Visualizing our features

This section will have visualizations which will be created automatically
        based on rules assigned for the type of variable being visualized. 

Rules for single variable visualizations:
* Numerical variables will be represented by histograms.
* The visualizations for numerical variables will have "Frequency" as the y-axis label.
* Categorical variables will be represented by bar charts.
* The visualizations for categorical variables will have "Count" as the y-axis label. 
            

In [143]:
def plot_single_feature(df, feature):
    """
    This function will be used to plot a single feature.

    Every feature's type will be first evaluated and then the
    feature's distribution will be graphed accordingly.

    Rules for single variable visualizations:
    * Numerical variables will be represented by histograms.
    * The visualizations for numerical variables will have "Frequency" as the y-axis label.
    * Categorical variables will be represented by bar charts.
    * The visualizations for categorical variables will have "Count" as the y-axis label. 

    Parameters
    ----------
    df: DataFrame
        A dataframe containing the heart disease data.
        
    feature: str
        The feature whose data needs to be plotted.

    Returns
    -------
    None
    """
    fig = None
    xaxis_type=None
    yaxis_title=""

    # Switching int features with low cardinality to object:
    df["num_major_vessels"] = df["num_major_vessels"].astype("object")
    df["target"] = df["target"].astype("object")
    # Check feature type and plot appropriately:
    if df[feature].dtype == 'int64' or df[feature].dtype == 'float64':
        #TODO(Sayar) Add slider widget here:
        fig = px.histogram(x=df[feature].values, nbins=0)

        yaxis_title = "Frequency"

    elif df[feature].dtype == 'object':
        fig = px.bar(y=df[feature].value_counts(), 
                     x=df[feature].value_counts().index.astype(str), 
                     color=df[feature].value_counts().index.astype(str), 
                     text=df[feature].value_counts())

        xaxis_type = "category"
        yaxis_title = "Count"

    fig.update_xaxes(title=feature)
    fig.update_yaxes(title=yaxis_title)
    fig.update_layout(showlegend=False, 
                      title="Distribution of {}".format(feature), 
                      xaxis_type=xaxis_type)
    fig.show()

    return 

In [144]:
# Iterating over every feature and plotting it:
for feature in df.columns:
    plot_single_feature(df, feature)

We would also like to plot interactions between two features to understand the relationship between them. Let's try to do this. We can again leverage writing a function to make things easier for us to do.

Feature interaction visualizations will have two variables and will plot the relationship between them.

Rules for feature interaction visualization:
* Only two variables can be used for this visualization.
* Both variables have to be different.
* For numerical vs numerical visuals, we will be using scatter plots.
* For numerical vs categorical visuals, we will be using box plots.
* For categorical vs categorical visuals, we will be using scatter plots.

In [4]:
def plot_numerical_numerical(df, feature_1, feature_2):
    """Plots numerical vs numerical features"""
    fig = px.scatter(df, feature_1, feature_2)
    fig.update_layout(title="Plot of {} vs. {}".format(feature_1, 
                                                       feature_2))
    fig.show()

def plot_numerical_categorical(df, feature_1, feature_2):
    """Plots numerical vs categorical features"""
    x_var, y_var = feature_1, feature_2
    # feature_1 is passed into x_var. If it is not categorical, 
    # we switch it with y_var:
    if df[feature_1].dtypes == "int64" or df[feature_1].dtypes == "float64":
        x_var,y_var = y_var,x_var

    fig = px.box(df, 
                 x=x_var, 
                 y=y_var, 
                 color=x_var)
                 
    fig.update_layout(title="Plot of {} vs. {}".format(x_var, y_var))

    fig.show()

def plot_categorical_categorical(df, feature_1, feature_2):
    """Plots categorical vs categorical features"""
    fig = px.parallel_categories(df, 
                                 dimensions=[feature_1, feature_2], 
                                 )
    fig.update_layout(title="Plot of {} vs. {}".format(feature_1, feature_2))
    fig.show()

def plot_dual_features(df, feature_1, feature_2):
    """
    This function will be used to plot feature interactions between
    two features.

    Rules for feature interaction visualization:

    * Only two variables can be used for this visualization.
    * Both variables have to be different.
    * For numerical vs numerical visuals, we will be using scatter plots.
    * For numerical vs categorical visuals, we will be using box plots.
    * For categorical vs categorical visuals, we will be using scatter plots.

    Parameters
    ----------
    df: DataFrame
        A dataframe containing the heart disease data.
    
    feature_1: str
        The first feature to be used in the plot.

    feature_2: str
        The second feature to be used in the plot.

    Returns
    -------
    None
    """
    # Cannot allow same feature plots:
    if feature_1 == feature_2:
        raise ValueError("Please select two different features.")

    # Changed to object type because of low cardinality:
    df["num_major_vessels"] = df["num_major_vessels"].astype("object")
    df["target"] = df["target"].astype("object")
    feature_1_type = str(df[feature_1].dtype)
    feature_2_type = str(df[feature_2].dtype)

    # Dictionary to hash the appropriate function object:
    switch_dict = {
        ("int64", "float64"): plot_numerical_numerical, 
        ("float64", "int64"): plot_numerical_numerical,
        ("float64", "float64"): plot_numerical_numerical,
        ("int64", "int64"): plot_numerical_numerical,
        ("int64", "object"): plot_numerical_categorical,
        ("float64", "object"): plot_numerical_categorical,
        ("object", "int64"): plot_numerical_categorical,
        ("object", "float64"): plot_numerical_categorical,
        ("object", "object"): plot_categorical_categorical
    }

    # Calling function object:
    switch_dict[(feature_1_type, feature_2_type)](df, feature_1, feature_2)

    return

Let's test to see this works properly!

In [3]:
plot_dual_features(df, "sex", "num_major_vessels")
plot_dual_features(df, "age", "cholesterol")
plot_dual_features(df, "st_slope", "thalassemia")

NameError: name 'plot_dual_features' is not defined

## Model to predict the presence of heart disease

In [147]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

Here are a few things we need to do before we proceed:
* We must dummify the categorical variables before we proceed with building the Machine  Learning model. 
* We must also ensure to remove the target variable from the rest of the data.
* Lastly, we must change the target and the num_major_vessels to integers.

In [148]:
target = df['target']

# Changing back low cardinality variables to int:
target = target.astype(int)
df['num_major_vessels'] = df['num_major_vessels'].astype(int)

# Dropping target from data:
df.drop('target', axis=1, inplace=True)

In [149]:
# Converting categorical variables into dummy variables:
df = pd.get_dummies(df, drop_first=True)

In [150]:
# Sanity check:
df.head()

,age,resting_blood_pressure,cholesterol,max_heart_rate_achieved,st_depression,num_major_vessels,sex_male,chest_pain_type_atypical angina,chest_pain_type_non-anginal pain,chest_pain_type_typical angina,fasting_blood_sugar_lower than 120mg/ml,rest_ecg_left ventricular hypertrophy,rest_ecg_normal,exercise_induced_angina_yes,st_slope_flat,st_slope_upsloping,thalassemia_normal,thalassemia_reversable defect
0,63,145,233,150,2.3,0,1,0,0,0,0,0,1,0,0,1,0,0
1,37,130,250,187,3.5,0,1,0,1,0,1,0,0,0,0,1,0,1
2,41,130,204,172,1.4,0,0,1,0,0,1,0,1,0,0,0,0,1
3,56,120,236,178,0.8,0,1,1,0,0,1,0,0,0,0,0,0,1
4,57,120,354,163,0.6,0,0,0,0,1,1,0,0,1,0,0,0,1


In [151]:
# Splitting into train and test:
X_train, X_test, y_train, y_test = train_test_split(df, 
                                                    target, 
                                                    test_size=0.2, 
                                                    random_state=42)

# Checking shapes:
print("X_train shape: {}".format(X_train.shape))
print("X_train shape: {}".format(X_test.shape))
print("y_train shape: {}".format(y_train.shape))
print("y_test shape: {}".format(y_test.shape))

X_train shape: (242, 18)
X_train shape: (61, 18)
y_train shape: (242,)
y_test shape: (61,)


### Random Forest Classiciation

In [152]:
# Initializing Random Forest model:
model = RandomForestClassifier(n_estimators=100, 
                               max_depth=4)

# Fitting the model:
model.fit(X_train, y_train)

# Predicting outcomes:
y_pred = model.predict(X_test)

# Checking accuracy:
print("Training accuracy: {}".format(model.score(X_train, y_train)))
print("Testing accuracy: {}".format(model.score(X_test, y_test)))

report = classification_report(y_test, y_pred)
print(report)

Training accuracy: 0.8842975206611571
Testing accuracy: 0.8852459016393442
              precision    recall  f1-score   support

           0       0.87      0.90      0.88        29
           1       0.90      0.88      0.89        32

    accuracy                           0.89        61
   macro avg       0.88      0.89      0.89        61
weighted avg       0.89      0.89      0.89        61



### Sensitivity and Specificity

In [153]:
conf_matrix = confusion_matrix(y_test, y_pred)
total = conf_matrix.sum()

# Computing sensitivity and specificity:
sensitivity = conf_matrix[0, 0]/(conf_matrix[0, 0] + conf_matrix[1, 0])
specificity = conf_matrix[1, 1]/(conf_matrix[1, 1] + conf_matrix[1, 0])

print("Sensitivity: {}".format(sensitivity))
print("Specificity: {}".format(specificity))

Sensitivity: 0.8666666666666667
Specificity: 0.875


## Saving the model

We must save the model so that we can use it within our Streamlit app for making predictions

In [154]:
import joblib

In [155]:
joblib.dump(model, "../models/rf_model.bin")

['../models/rf_model.bin']

## Next Steps

Here I have outlined some next steps for improving the model, inference and explainability:
* Perform more feature engineering
* Use shap values for model explainability
* Plot the model tree structure.
* Check impact of each feature on the model using partial dependence and permutation importance.

Thank you for attending this workshop session! I hope all of you got to learn something new! 